In [ ]:
import os
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

import pyspark

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep='\t', header=True,inferSchema=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars|Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|    

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame

In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupBy("customer_id").count()
customers_df = customers_df.withColumnRenamed('count','customer_count')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    9015608|             3|
|    9767880|             2|
|   15958313|             1|
|   12879980|            20|
|   43920023|            31|
|     106535|             1|
|   20328662|             1|
|   12819130|             4|
|   50145309|             1|
|   15159322|             2|
|   16747249|             2|
|   37678162|             1|
|     227763|             1|
|   15191144|             2|
|   31946913|             1|
|   44295916|             1|
|   19021463|             1|
|     806337|             2|
|   16306618|            18|
|     393186|             1|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id', 'product_title']).drop_duplicates(['product_id'])
products_df = products_df.distinct()
products_df.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0451192362|how stella got he...|
|0594169909|Pink Wide Stripe ...|
|0702001996|  Mastiha Hand Cream|
|0737104473|Hello Kitty Lustr...|
|0785374426|Charlotte Russe R...|
+----------+--------------------+
only showing top 5 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3I2DHQBR577SS|    1797882|B001ANOOOE|       2102612| 2015-08-31|
|R1QNE9NQFJC2Y4|   18381298|B0016J22EQ|     106393691| 2015-08-31|
|R3LIDG2Q4LJBAO|   19242472|B00HU6UQAG|     375449471| 2015-08-31|
|R3KSZHPAEVPEAL|   19551372|B002HWS7RM|     255651889| 2015-08-31|
| RAI2OIG50KZ43|   14802407|B00SM99KWU|     116158747| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show(5)
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn('star_rating',df.star_rating.cast(IntegerType()))
vine_df.schema

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



StructType(List(StructField(review_id,StringType,true),StructField(star_rating,IntegerType,true),StructField(helpful_votes,IntegerType,true),StructField(total_votes,IntegerType,true),StructField(vine,StringType,true),StructField(verified_purchase,StringType,true)))

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.ch6ylfrhb3uz.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "JToews19", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write review_id_df to table in RDS
#review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


Py4JJavaError: ignored

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
#customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)